## Preprocessing if needed

In [1]:
from parselmouth.praat import call, run_file
import pyloudnorm as pyln
import noisereduce as nr
import tensorflow as tf
import soundfile as sf
from glob import glob
import numpy as np
import parselmouth 
import librosa
import random
import re
import os


In [2]:
#path for temporary audio files
tmp_path = "D:\\Tonaufnahmen\\tmp\\tmp.wav"

#path of own praat scripts
praat_marksyllables = "C:\\Users\\Lea\\Praat\\plugin_VocalToolkit\\marksyllables_own.praat"
praat_changeformants = "C:\\Users\\Lea\\Praat\\plugin_VocalToolkit\\changeformants_own.praat"

#path of the parent folder of the audio files
#DATASET_PATH = "D:\\Tonaufnahmen\\NEUERHEBUNG_WS_ALT_Sorted_Preprocessed_min05_AllOverThree - Kopie"
DATASET_PATH = "D:\\Tonaufnahmen\\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\\"

#length of one audio segment in seconds
length = 10.0

### Preprocessing functions

In [3]:
#https://superkogito.github.io/blog/2020/04/30/rms_normalization.html
def rms_normalize(sig, rms_level=0):
    r = 10**(rms_level / 10.0)
    a = np.sqrt( (len(sig) * r**2) / np.sum(sig**2) )
    y = sig * a
    return y


In [4]:
def peak_normalize(sig):
    y = sig / (max(sig)+0.2)
    return y


In [5]:
#https://medium.com/@poudelnipriyanka/audio-normalization-9dbcedfefcc0
def rms_normalize_pln(sig, rms_level=0):
    meter = pyln.Meter(16000)
    loudness = meter.integrated_loudness(sig)
    y = pyln.normalize.loudness(sig, loudness, rms_level)
    return y


In [6]:
#https://medium.com/@poudelnipriyanka/audio-normalization-9dbcedfefcc0
def peak_normalize_pln(sig, peak_level=0):
    y = pyln.normalize.peak(sig, peak_level)
    return y


In [7]:
def dc_offset(sig):
    y = sig - np.mean(sig)
    return y


In [8]:
def dc_offset_disturb(sig):
    disturbance = np.random.uniform(-0.1, 0.1)
    y = sig + disturbance
    return y


In [9]:
#normalize formants F1, F2 and F3
def normalize_formants(sig):
    path = tmp_path
    sf.write(path, sig, 16000, subtype='PCM_16')
    sound = parselmouth.Sound(path)
    new_sound = run_file(sound, praat_changeformants, 500, 1500, 25000, 0, 0, 5000, 'yes', 'yes', 'no')
    return new_sound[0].values.flatten()
        

In [10]:
#works better with Loudness/peak normalized audios
def get_new_tempo(audio, start, end):
    y, sr = librosa.load(new_path, sr=16000, res_type='soxr_vhq')
    sig = y[start:end]
    path = tmp_path
    sf.write(path, sig, 16000, subtype='PCM_16')
    sound = parselmouth.Sound(path)
    syllables = run_file(sound, praat_marksyllables, -25, 0.3, 2, 'yes', 'yes', capture_output=True)
    pattern = r"Articulation rate \(number of syllables \/ phonation time\): (\d+\.?\d*)"
    match = re.search(pattern, str(syllables))
    if match:
        articulation_rate = match.group(1)
        ar = float(articulation_rate)
        if (ar > 2 and ar < 6.5):
            #random_ar = random.uniform(2.5, 6)
            return ar/6.0
        else:
            print(syllables)
            return 1.0
    else:
        print("Pattern not found")
        

In [11]:
def make_new_tempo(sig, tempo):
    path = tmp_path
    sf.write(path, sig, 16000, subtype='PCM_16')
    sound = parselmouth.Sound(path)
    new_sound = call(sound, "Lengthen (overlap-add)", 75, 300, tempo)
    return new_sound.values.flatten()
    

In [12]:
def pitch_normalize(sig):
    new_pitch = 120
    #new_pitch = random.randint(90, 160)
    path = tmp_path
    sf.write(path, sig, 16000, subtype='PCM_16')
    sound = parselmouth.Sound(path)
    new_sound = call(sound, "Change gender", 80, 170, 1.0, new_pitch, 0.0, 1.0)
    return new_sound.values.flatten()
    

In [13]:
def pitch_normalize2(sig):
    new_pitch = 120
    new_std = 18
    path = tmp_path
    sf.write(path, sig, 16000, subtype='PCM_16')
    sound = parselmouth.Sound(path)
    
    pitch = call(sound, "To Pitch", 0.0, 80.0, 170.0)
    std_pitch = call(pitch, "Get standard deviation", 0.0, 0.0, 'Hertz', return_string=True)
    pattern = r"(\d+\.?\d*) Hz"
    match = re.search(pattern, str(std_pitch))
    if match:
        std_pitch_res = match.group(1)
        faktor = new_std/float(std_pitch_res)
    else:
        print(std_pitch)
        print("Pattern not found")
        return 0.0, 0  
    
    new_sound = call(sound, "Change gender", 80, 170, 1.0, new_pitch, faktor, 1.0)
    return new_sound.values.flatten()


In [14]:
def noise_reduction(sig):
    path = tmp_path
    sf.write(path, sig, 16000, subtype='PCM_16')
    sound = parselmouth.Sound(path)
    new_sound = call(sound, "Remove noise", 0, 0, 0.025, 80.0, 8000.0, 40.0, 'spectral-subtraction')
    return new_sound.values.flatten()


In [15]:
def noise_reduction2(sig):
    reduced_noise = nr.reduce_noise(y=sig, sr=16000)
    return reduced_noise


In [16]:
def preemphasis(sig):
    path = tmp_path
    sf.write(path, sig, 16000, subtype='PCM_16')
    sound = parselmouth.Sound(path)
    new_sound = call(sound, "Filter (pre-emphasis)...", 1300.807) #1300.807=0.6 #130.617=0.95
    return new_sound.values.flatten()

In [17]:
def cubic_root_compression(sig):
    #compressed_signal = np.sign(sig) * np.cbrt(np.abs(sig))
    compressed_signal = np.sign(sig) * (np.abs(sig) ** (2/3))
    return compressed_signal

### Audio Files in the DATASET_PATH will be replaced by preprocessed ones!!!

#### Output:

This code snippet preprocesses the audio data by resampling each audio file to a sampling rate of 16kHz, converting them to mono, concatenating multiple audio files into one, and saving them back to their original directory structure. The resulting audio files are saved with a bit-depth of 16.

Additionally, if there are multiple audio files within a single folder for a speaker, they are concatenated into a single audio file before preprocessing.

Preprocessing steps can be applied before saving the audio files.


In [18]:
all_classes = glob(DATASET_PATH + '\\*', recursive = True)
all_speaker = []

# list of all classes
for dialect in all_classes:
    all_speaker = np.concatenate((all_speaker, glob(dialect + '\\*', recursive = True)), axis=None)
    
for path in all_speaker:
    audios = tf.io.gfile.glob(path + '\\*.wav')
    one_audio = []
    print(path)
    for audio in audios:
        # sample to 16kHz
        y, sr = librosa.load(audio, sr=16000, res_type='soxr_vhq')
        # to mono
        librosa.to_mono(y)
        # put all Audios of one speaker into one File
        one_audio.append(y)
        # remove original File
        os.remove(audio)
    if len(one_audio) < 1:
        print('Speaker empty')
    else:
        one_audio = np.concatenate(one_audio).ravel()
        if len(one_audio) < length*sr:
            print('Speaker to short!')
        else:
            #Preprocessing
            res_audio = []
            sr_length = int(length*16000)
            times = len(one_audio)//(sr_length)
            for i in range(0, times):
                ad = one_audio[i*sr_length:(i+1)*sr_length]
                #preprocessing step
                #tempo = get_new_tempo(audio, i*sr_length, (i+1)*sr_length)
                y = peak_normalize(ad)
                #y = normalize_formants(ad)
                #y1 = preemphasis(ad)
                #y = cubic_root_compression(y1)
                #y = ad
                res_audio.append(y)
                sf.write(path + '\\' + path.split('\\')[-1] + "_" + str(i) + '.wav', y, 16000, 'PCM_16')
            res_audio = np.concatenate(res_audio).ravel()
            # save as wav with a sampling-rate of 16kHz and bit-depth of 16
            #sf.write(path + '\\' + path.split('\\')[-1] + '.wav', res_audio, 16000, 'PCM_16')


D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Brandenburgisch\BRB1
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Brandenburgisch\BRB2
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Brandenburgisch\BRB3
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Brandenburgisch\BRBALT1
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Brandenburgisch\BRBALT2
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Brandenburgisch\BRBJUNG3
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Brandenburgisch\OHV1
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Brandenburgisch\OPR1
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Brandenburgisch\OPRALT1
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05

D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Mittelalemannisch\RV2
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Mittelalemannisch\RV3
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Mittelalemannisch\RVALT
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Mittelalemannisch\RVJUNG1
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Mittelalemannisch\TUT1
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Mittelalemannisch\TUT3
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Mittelalemannisch\TUTALT2
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Mittelalemannisch\TUTJUNG1
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Mittelalemannisch\VS3
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Prepr

D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Niederfraenkisch\OB2
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Niederfraenkisch\OB6
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Niederfraenkisch\OBALT1
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Niederfraenkisch\OBALT2
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Nordbairisch\AM2
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Nordbairisch\AM3
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Nordbairisch\AMALT1
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Nordbairisch\AMJUNG1
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Nordbairisch\AMJUNG2
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - 

D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Nordniederdeutsch\PIJUNG1
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Nordniederdeutsch\RD5
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Nordniederdeutsch\RD7
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Nordniederdeutsch\RDALT
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Nordniederdeutsch\RDJUNG1
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Nordniederdeutsch\RDJUNG2
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Nordniederdeutsch\SE1
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Nordniederdeutsch\SE2
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Nordniederdeutsch\SE5
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preproce

D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Ostfraenkisch\ERALT2
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Ostfraenkisch\ERJUNG1
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Ostfraenkisch\HBN1
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Ostfraenkisch\HBN2
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Ostfraenkisch\HBNALT1
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Ostfraenkisch\HBNALT2
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Ostfraenkisch\HBNJUNG1
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Ostfraenkisch\HO1
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Ostfraenkisch\HO2
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Ko

D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Ripuarisch\AC4
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Ripuarisch\ACALT1
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Ripuarisch\ACALT2
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Ripuarisch\ACJUNG1
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Ripuarisch\BM2
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Ripuarisch\BM3
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Ripuarisch\BMALT2
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Ripuarisch\BMALT3
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Ripuarisch\BMJUNG1
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Ripuarisch\DN1
D:\Tonaufn

D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Thueringisch\EF3
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Thueringisch\EFALT1
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Thueringisch\EFALT2
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Thueringisch\EFJUNG1
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Thueringisch\EFJUNG2
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Thueringisch\EIC1
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Thueringisch\EIC4
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Thueringisch\EICALT1
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Thueringisch\EICALT2
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Thu

D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Zentralhessisch\GI11
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Zentralhessisch\GI12
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Zentralhessisch\GIALT1
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Zentralhessisch\GIALT2
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Zentralhessisch\GIJUNG
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Zentralhessisch\HGALT1
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Zentralhessisch\LDK2
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Zentralhessisch\LDK3
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_AllOverThree - Kopie\Zentralhessisch\LDKALT1
D:\Tonaufnahmen\NEUERHEBUNG_WS_ALL_Sorted_Preprocessed_min05_A